In [1]:
import os
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [2]:
# import tensorflow as tf
# print(tf.__version__)
# print(tf.config.list_physical_devices('GPU'))
# print(tf.test.is_built_with_cuda())

## Download dataset 
https://drive.google.com/drive/u/3/folders/1sHh6NvuKX6RB5OytLwf4kaqfQ9svJNDQ

## Load data

In [3]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")

x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [4]:
# It's a multi-class classification problem 
class_index = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4,
               'dog': 5, 'frog': 6,'horse': 7,'ship': 8, 'truck': 9}
print(np.unique(y_train))

[0 1 2 3 4 5 6 7 8 9]


![image](https://img-blog.csdnimg.cn/20190623084800880.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3lqcDE5ODcxMDEz,size_16,color_FFFFFF,t_70)

## Data preprocess

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to one-hot encoding (keras model requires one-hot label as inputs)
num_classes = 10
print(y_train[0])
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train[0])

[9]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


## Build model & training (Keras)

In [6]:
# # Build model
# model = Sequential() # Sequential groups a linear stack of layers 
# model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=x_train.shape[1:])) # Add Convolution layers
# model.add(Activation('relu')) # Add Relu activation for non-linearity
# model.add(Conv2D(filters=32, kernel_size=(3, 3))) # Add Convolution layers
# model.add(Activation('relu')) # Add Relu activation for non-linearity
# model.add(MaxPooling2D(pool_size=(4, 4))) # Add Max pooling to lower the sptail dimension

# model.add(Flatten()) # Flatten the featuremaps
# model.add(Dense(units=512)) # Add dense layer with 512 neurons
# model.add(Activation('relu')) # Add Relu activation for non-linearity
# model.add(Dense(units=num_classes)) # Add final output layer for 10 classes
# model.add(Activation('softmax')) # Add softmax activation to transfer logits into probabilities

# # initiate SGD optimizer
# opt = keras.optimizers.SGD()

# # Compile the model with loss function and optimizer, and evaluate with accuracy
# model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# # Setup some hyperparameters
# batch_size = 32
# epochs = 10

# # Fit the data into model
# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_data=(x_test, y_test),
#           shuffle=True)


In [7]:
# # Build Model
# model = Sequential()
# model.add(Conv2D(filters=64, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
# model.add(Conv2D(filters=64, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
# # model.add(MaxPooling2D(pool_size=2))

# model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
# model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
# # model.add(MaxPooling2D(pool_size=2))

# model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
# model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
# # model.add(MaxPooling2D(pool_size=2))


# model.add(Flatten()) # Flatten the featuremaps
# model.add(Dense(units=512, activation='relu')) # Add dense layer with 512 neurons
# model.add(Dense(units=num_classes, activation='softmax')) # Add final output layer for 10 classes

In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=num_classes, activation='softmax'))

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)       

In [10]:
# initiate SGD optimizer
opt = keras.optimizers.SGD()

# Compile the model with loss function and optimizer, and evaluate with accuracy
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Setup some hyperparameters
batch_size = 32
epochs = 30
data_augmentation = True


In [11]:
if not data_augmentation:
    # Fit the data into model
    model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)
else:
    print('Apply data augmentation...')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        rescale=0.9,
        rotation_range=3,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

Apply data augmentation...
Epoch 1/30
1563/1563 [==============================] - 22s 14ms/step - loss: 2.0934 - accuracy: 0.2226 - val_loss: 1.8063 - val_accuracy: 0.3610
Epoch 2/30
1563/1563 [==============================] - 21s 13ms/step - loss: 1.7762 - accuracy: 0.3559 - val_loss: 1.6187 - val_accuracy: 0.4111
Epoch 3/30
1563/1563 [==============================] - 21s 13ms/step - loss: 1.6371 - accuracy: 0.4037 - val_loss: 1.6187 - val_accuracy: 0.4175
Epoch 4/30
1563/1563 [==============================] - 21s 13ms/step - loss: 1.5271 - accuracy: 0.4418 - val_loss: 1.5066 - val_accuracy: 0.4611
Epoch 5/30
1563/1563 [==============================] - 21s 13ms/step - loss: 1.4478 - accuracy: 0.4736 - val_loss: 1.3825 - val_accuracy: 0.5127
Epoch 6/30
1563/1563 [==============================] - 21s 13ms/step - loss: 1.3824 - accuracy: 0.5002 - val_loss: 1.2930 - val_accuracy: 0.5415
Epoch 7/30
1563/1563 [==============================] - 21s 13ms/step - loss: 1.3292 - accuracy: 

In [12]:
y_pred = model.predict(x_test)
print(y_pred.shape) # 10000 samples, each sample with probaility of 10 classes

(10000, 10)


In [13]:
# y_pred[0] 

In [14]:
# np.argmax(y_pred[0]) # argmax to find the predict class with highest probability. 9=truck

In [15]:
# y_pred = np.argmax(y_pred, axis=1)

## DO NOT MODIFY CODE BELOW!
**Please screen shot your results and post it on your report**

In [16]:
# y_pred = your_model.predict(x_test)
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

In [17]:
assert y_pred.shape == (10000,)

In [18]:
y_test = np.load("y_test.npy")
print("Accuracy of my model on test set: ", accuracy_score(y_test, y_pred))

Accuracy of my model on test set:  0.7602
